In [5]:
import requests
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs

from tqdm import tqdm

import time
import datetime

import pandas as pd

In [126]:
def doScrollDown(driver, scrolling_cnt):
    cnt = 0
    current_scroll = 0 
    while cnt < scrolling_cnt:  # 최대 10번 스크롤
        max_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script(f'window.scrollTo({current_scroll}, {max_height-3});')
        
        driver.implicitly_wait(5) #페이지 로드 완료까지 대기, 완료되면 넘어감
        
        # 더이상 가져올 리뷰가 없으면 반복문 종료
        if current_scroll == max_height: break
        
        current_height = max_height
        cnt+=1

In [127]:
def crawling_top_reviewer():
    url = 'https://www.oliveyoung.co.kr/store/mypage/getReviewerLounge.do'
    driver = webdriver.Chrome("./driver/chromedriver 2")
    driver.get(url) # 메인 페이지
    driver.implicitly_wait(5)

    review_dic = dict()

    cnt = 1
    page = 1

    while True:

        driver.find_element(By.XPATH, f'//*[@id="topReviewerList"]/li[{cnt}]/div/a').click()
        driver.implicitly_wait(5)

        # 스크롤 부분
        scrolling_cnt = 1 # 기본 리뷰 10개. 1당 10개씩 증가. [0-> 10개 , 10 -> 110개]
        doScrollDown(driver, scrolling_cnt)

        tmp = dict()

        grade = driver.find_element(By.XPATH, '//*[@id="Contents"]/div/div[1]/div[1]/div/div[1]/div[2]/div[2]/span/a').text

        tmp['nickname'] = driver.find_element(By.XPATH, '//*[@id="Contents"]/div/div[1]/div[1]/div/div[1]/div[2]/div[1]/p/strong').text
        tmp['help_cnt'] = driver.find_element(By.XPATH, '//*[@id="recommedCnt"]').text
        tmp['follower'] = driver.find_element(By.XPATH, '//*[@id="followerCount"]').text
        tmp['folowing'] = driver.find_element(By.XPATH, '//*[@id="followingCount"]').text

        html = (driver.page_source)
        soup = bs(html, "html.parser")
        tmp['reviews'] = soup.select('#reviewerGdasList > div')

        review_dic[grade] = tmp

        driver.back() # 전 페이지로 이동
        driver.implicitly_wait(5) #페이지 로드 완료까지 대기, 완료되면 넘어감
        
        # 페이지 넘아가기 & 아이템 순번 초기화
        if cnt == 10:
            if page == 2:
                driver.find_element(By.XPATH, '//*[@id="Container"]/div[2]/div/div[2]/a[2]').click()
            else:
                driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div/div[2]/a[{page}]').click()
            cnt = 0
            page+=1
        
        # 작동확인을 위한 코드 삭제하면 끝까지 삭제됨. 원하는 페이지 만큼 삭제 가능 페이지당 리뷰어 10명
        if page > 4 :
            break

        cnt+=1
        
    return review_dic


In [128]:
import re
def make_df(datasets):
    result = pd.DataFrame()
    
    for key, data in datasets.items():
        reviewer = pd.DataFrame()
        for review in data['reviews']:
            
            item_name = (review.select_one('div > img')['alt'])
            
            item_review_point = (review.select_one('span.point').text)
            item_review_point = re.search(' [0-9](?=점)', item_review_point).group(0)
            
            item_review_writed_at = review.select_one('span.review_point_text').text
            item_review_writed_at = re.search('[0-9]+[.][0-9]+[.]+[0-9]+', item_review_writed_at).group(0)
            
            item_buy_offline = review.select_one('span.ico_offlineStore').text if review.select_one('span.ico_offlineStore') else ''            
            item_buy_offline = 1 if item_buy_offline == '매장' else 0
            
            item_review_content = review.select_one('p.rw-box__description').text
            
            item_review_recommand = review.select_one('span.num').text.strip()
                
            
            tmp = pd.DataFrame({
                'item' : item_name,
                'content' : item_review_content,
                'review_recommand' : item_review_recommand,
                'review_point' : item_review_point,
                'writed_at' : item_review_writed_at,
                'buyOffline' : item_buy_offline
            }, index=[0])
            
            reviewer = pd.concat([reviewer, tmp])
        
        reviewer['reviewer'] = data['nickname']
        
        grade = list(data.keys())[0]
        reviewer['reviewer_grade'] = re.sub('TOP','', key)
        
        
        result = pd.concat([result, reviewer])
            
    return result.reset_index(drop=True)
        

In [129]:
data = crawling_top_reviewer()
review_df = make_df(data)

/var/folders/jt/jw2pb8hx71d48w0rx0cgqf_80000gn/T/ipykernel_13248/101017196.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver 2")


In [130]:
review_df

,item,content,review_recommand,review_point,writed_at,buyOffline,reviewer,reviewer_grade
0,센트룸 멀티비타민 실버 맨 50정 (50일분),신랑은 센트룸 실버맨 저는 실버우먼을\n먹고 있어요ㅎ\n하루 1알만 먹으면 되고 한...,0,5,2023.04.18,1,어피치콜,1
1,[선우PICK]메디힐 티트리 진정 패드 더블 기획 (100매+100매 리필),여름이면 피부가 붉어져서 티트리 진정패드를\n많이 사용을 해서 날씨가 더워질때쯤 많...,0,5,2023.04.18,1,어피치콜,1
2,HBAF 메이플믹스넛 30g,올리브영에서 1+1 행사를 해서 자주\n구매를 하고 있는데 신랑이 하루에 두봉지씩\...,0,5,2023.04.18,1,어피치콜,1
3,낫띵베럴 이너주스 사과당근 120ml,팽이버섯 유산균발효액이 들어가서\n꾸준하게 마시면 화장실 가는게 편해\n지더라구요😁,0,5,2023.04.17,1,어피치콜,1
4,[컬러브로우 마스카라] 키스미 헤비로테이션 컬러링 브로우,컬러는 여러가지라서 본인의\n머리카락 색이랑 비슷한걸로\n선택하면 되요ㅎㅎ\n눈썹에...,0,5,2023.04.17,1,어피치콜,1
...,...,...,...,...,...,...,...,...
795,햇반솥반 불고기버섯영양밥 200G,전자레인지에 조리해서 먹을 수 있는 영양밥입니다\n재료가 다양해서 좋아요,0,5,2023.04.12,0,올리브주아,40
796,햇반솥반 불고기버섯영양밥 200G,양이 한끼로 딱 좋아요\n버섯과 고기 당근이 들어 있어서 식감도 좋아요,0,5,2023.04.12,0,올리브주아,40
797,햇반솥반 불고기버섯영양밥 200G,영양밥이라서 기숙사에서 먹기 좋아요\n배달 음식 시킬 때 같이 먹어도 좋아요,0,5,2023.04.12,0,올리브주아,40
798,미쟝센 살롱10 단백질 트리트먼트 (손상) 120ml,손상모에 사용할 수 있는 트리트먼트입니다\n향이 좋아서 잘 사용하고 있어요,0,5,2023.04.12,0,올리브주아,40
